<a href="https://colab.research.google.com/github/tumurzakov/latentflow/blob/main/demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install diffusers transformers einops git+https://github.com/openai/CLIP.git \
  torchvision compel mediapy OmegaConf accelerate xformers decord[torch] pytorch_lightning \
  basicsr facenet-pytorch rembg open-clip-torch kornia
  
!git clone https://github.com/tumurzakov/latentflow.git
!git clone https://github.com/tumurzakov/AnimateDiff.git
!git clone https://github.com/comfyanonymous/ComfyUI.git
!git clone https://github.com/xinntao/Real-ESRGAN.git /content/RESRGAN
!touch /content/RESRGAN/realesrgan/version.py

!wget -c "https://huggingface.co/guoyww/animatediff/resolve/main/mm_sd_v15_v2.ckpt?download=true" -O "/content/mm_sd_v15_v2.ckpt"

!git lfs install
!git clone https://huggingface.co/runwayml/stable-diffusion-v1-5 --branch fp16 --depth 1


In [ ]:
%%writefile /content/inference.yaml

unet_additional_kwargs:
  unet_use_cross_frame_attention: false
  unet_use_temporal_attention: false
  use_motion_module: true
  motion_module_resolutions:
  - 1
  - 2
  - 4
  - 8
  motion_module_mid_block: false
  motion_module_decoder_only: false
  motion_module_type: Vanilla
  motion_module_kwargs:
    num_attention_heads: 8
    num_transformer_block: 1
    attention_block_types:
    - Temporal_Self
    - Temporal_Self
    temporal_position_encoding: true
    temporal_position_encoding_max_len: 32
    temporal_attention_dim_div: 1

noise_scheduler_kwargs:
  DDIMScheduler:
    num_train_timesteps: 1000
    beta_start: 0.00085
    beta_end: 0.012
    beta_schedule: "linear"
    steps_offset: 1
    clip_sample: false

  EulerAncestralDiscreteScheduler:
    num_train_timesteps: 1000
    beta_start: 0.00085
    beta_end: 0.012
    beta_schedule: "linear"

  KDPM2AncestralDiscreteScheduler:
    num_train_timesteps: 1000
    beta_start: 0.00085
    beta_end: 0.012
    beta_schedule: "linear"


In [ ]:
import sys
if 'AnimateDiff' not in sys.path:
  sys.path.insert(0, '/content/latentflow/src')
  sys.path.insert(0, '/content/AnimateDiff')
  sys.path.insert(0, '/content/ComfyUI')
  sys.path.insert(0, '/content/RESRGAN')

In [ ]:
import latentflow as lf

pipe = lf.AnimateDiffPipeline.load(
    pretrained_model_path=f'{models}/stable-diffusion-v1-5',
    motion_module_path=f'{models}/mm_sd_v15_v2.ckpt',
    motion_module_config_path=f'{models}/mm_sd_v15_v2.yaml',
    scheduler_class_name='EulerDiscreteScheduler',
    fp16=False,
    xformers=False,
)

state = lf.State({
    'width': 512,
    'height': 288,
    'video_length': 16,
    'num_inference_steps': 20,
    'prompt': 'a cat walking down city street',
    'fps': 16,
})

video = (
    (lf.Latent()
        | pipe(
            num_inference_steps=state['num_inference_steps'],
            prompt=state['prompt'],
            video_length=state['video_length'],
            temporal_context=state['video_length'],
            width=state['width'],
            height=state['height'],
        )
        | lf.VaeLatentDecode(vae=pipe.vae, vae_batch=4)
        | lf.VideoShow(fps=state['fps'])
        )
)